# Ajuste de hiper-parámetros

Cuando trabajamos con modelos de *machine learning* a menudo nos encontramos con que estos disponen de una serie de hiper-parámetros que es necesario ajustar con el fin de lograr que realicen las predicciones más certeras posibles sobre nuestro conjunto de datos. A este paso se le conoce como **hyper-parameters tunning**.

Veamos cómo podemos ajustar los hiper-parámetros de un `RandomForestClassifier`para el conjunto de datos de las caras de Olivetti.

Cargamos las caras:

In [ ]:
from sklearn.datasets import fetch_olivetti_faces

olivetti = fetch_olivetti_faces()

El conjunto de datos de las caras de Olivetti busca identificar a 40 personas mediante imágenes de sus rostros.

In [ ]:
print(olivetti.DESCR)

Algunas de las caras:

In [ ]:
import matplotlib.pyplot as plt

n_images = 6
image_shape = (64, 64)

random_indices = np.random.choice(X.shape[0], size=n_images, replace=False)
faces = X[random_indices, :]

fig, axs = plt.subplots(nrows=1, ncols=n_images, figsize=(3*n_images, 3))

for i, comp in enumerate(faces):
    axs[i].imshow(comp.reshape(image_shape), cmap=plt.cm.gray, interpolation='nearest', vmin=0, vmax=1)
    axs[i].set_xticks(())
    axs[i].set_yticks(())

Prepramos el dataset y lo partimos en `train` y `test`:

In [ ]:
X = olivetti.data
y = olivetti.target

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Optimización de un hiper-parámetro

Si quieremos optimizar un único híper-parametro, lo ideal es analizar cómo se comporta una medida de calidad según evoluciona dicho hiper-parámetro.

Por ejemplo, veamos como varía el *accuracy* al variar el hiper-parámetro `n_estimators`:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier

n_estimators = np.arange(1,50,3)
train_scores = np.array([])
test_scores = np.array([])

for n in n_estimators:
  clf = RandomForestClassifier(n_estimators=n, random_state=42).fit(X_train, y_train)
  train_scores = np.append(train_scores, clf.score(X_train, y_train))
  test_scores = np.append(test_scores, clf.score(X_test, y_test))

plt.figure()

plt.plot(n_estimators, train_scores, label="train")
plt.plot(n_estimators, test_scores, label="test")

plt.xlabel('n_estimators')
plt.ylabel('accuracy')

plt.legend()

## Optimización de dos hiper-parámetros

Si queremos comprobar cómo evoluciona el error cuando para las variaciones producidas por dos hiper-parámetros, debemos pintar un mapa de calor en el que se muestre el resultado de evaluar el modelo para las combinaciones de calores de los dos hiper-parámetros.

Veamos como evoluciona el *accuracy* cuando variamos `n_estimators` y `max_depth`:

In [ ]:
import matplotlib.pyplot as plt

n_estimators = np.arange(5,51,5)
max_depths = np.arange(1,30,3)

train_scores = np.array([])
test_scores = np.array([])

for n in n_estimators:
  for d in max_depths:
    clf = RandomForestClassifier(n_estimators=n, max_depth=d, random_state=42).fit(X_train, y_train)
    train_scores = np.append(train_scores, clf.score(X_train, y_train))
    test_scores = np.append(test_scores, clf.score(X_test, y_test))

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5))

# train error 

im0 = axs[0].imshow(train_scores.reshape((len(n_estimators), len(max_depths))), vmin=0, vmax=1, cmap='Blues')
plt.colorbar(im0, ax=axs[0])

axs[0].set_xlabel('max_depths')
axs[0].set_xticks(np.arange(len(max_depths)))
axs[0].set_xticklabels(max_depths)

axs[0].set_ylabel('n_estimators')
axs[0].set_yticks(np.arange(len(n_estimators)))
axs[0].set_yticklabels(n_estimators)

axs[0].set_title('Train')

# test error

im1 = axs[1].imshow(test_scores.reshape((len(n_estimators), len(max_depths))), vmin=0, vmax=1, cmap='Blues')
plt.colorbar(im1, ax=axs[1])

axs[1].set_xlabel('max_depths')
axs[1].set_xticks(np.arange(len(max_depths)))
axs[1].set_xticklabels(max_depths)

axs[1].set_ylabel('n_estimators')
axs[1].set_yticks(np.arange(len(n_estimators)))
axs[1].set_yticklabels(n_estimators)

axs[1].set_title('Test')

## Optimización de múltiples hiper-parámetros

Para analizar qué sucede con más de dos hiper-parámetros, no podemos pintar ningún tipo de gráfico al tener más de tres dimensiona a representar (dos para los hiper-parámetros y una para el error). Por ello, en lugar de representar gráficamente el error lo hacemos de forma tabulada.

[`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) es una excelente herramienta que prueba de forma exahstiva todas las combinaciones de hièr-parámetros para unos rangos pre-seleccionados.

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
  'n_estimators': np.arange(5,51,10),
  'max_depth': np.arange(1,30,5),
  'criterion': ('gini', 'entropy')
}

rf =  RandomForestClassifier(random_state=42)
gs = GridSearchCV(rf, parameters, cv=3)
gs.fit(X, y)

In [ ]:
import pandas as pd
pd.DataFrame(gs.cv_results_).sort_values('rank_test_score')

El problema de *Grid Search* es que es profundamente explorativo, por lo que los tiempos de ejecución suelen dispararse. Como alternativa disponemos de [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) que selecciona aleatoriamente combinaciones de hiper-parámetros dentro del rango elegido.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
  'n_estimators': np.arange(5,51,5),
  'max_depth': np.arange(1,30,3),
  'criterion': ('gini', 'entropy')
}

rf =  RandomForestClassifier(random_state=42)
rs = RandomizedSearchCV(rf, parameters, n_iter=10)
rs.fit(X_train, y_train)

In [ ]:
import pandas as pd
pd.DataFrame(rs.cv_results_).sort_values('rank_test_score')

---

Creado por **Fernando Ortega** (fernando.ortega@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">